# Чатботы 2

В прошлый раз мы научились писать очень простого бота: он умеет получать сообщение и отправлять в ответ его же или, например, его длину. 

Но возможностей у ботов гораздо больше, и как ими пользоваться описано в [документации модуля](https://github.com/eternnoir/pyTelegramBotAPI#general-api-documentation).

## Обработчики сообщений
Мы с вами видели вот такой код:

In [ ]:
@bot.message_handler(commands=['start', 'help'])
def send_welcome(message):
	bot.send_message(message.chat.id, "Здравствуйте! Это бот, который считает длину вашего сообщения.")

Внутри декоратора `@bot.message_handler(...)` могут находиться фильтры, которые описывают, на какие сообщения реагирует данная функция. Фильтры пишутся так: сначала название фильтра, затем через знак равно его значение. Бывают фильтры четырех типов:
* `content_types`, значением является массив строк, задающих тип контента - текст, аудио, файл, стикер и т.д. (по умолчанию ['text'])
* `regexp`, значением является регулярное выражение (строка)
* `commands`, значением является массив строк, задающих команды без знака /
* `func`, значением является какая-то функция

Что бывает, когда боту приходит сообщение, которое подходит под несколько наших фильтров, т.е. несколько разных функций? В этом случае запускается функция, которая в вашем коде написана раньше других. 

[Примеры обработчиков](https://github.com/eternnoir/pyTelegramBotAPI#message-handlers)

# Отправка сообщений

Для отправки разных типов сообщений и разного контента используются разные функции.
Отправка текста:

In [ ]:
# ответить на сообщение (с цитированием)
tb.reply_to(message, text)

# отправить текст в чат по ID
tb.send_message(chat_id, text)

# переслать данное сообщение из одного чата в другой
tb.forward_message(to_chat_id, from_chat_id, message_id)

Отправка файлов (из документации):

In [ ]:
# All send_xyz functions which can take a file as an argument, can also take a file_id instead of a file.
# sendPhoto
photo = open('/tmp/photo.png', 'rb')
tb.send_photo(chat_id, photo)
tb.send_photo(chat_id, "FILEID")

# sendAudio
audio = open('/tmp/audio.mp3', 'rb')
tb.send_audio(chat_id, audio)
tb.send_audio(chat_id, "FILEID")

## sendAudio with duration, performer and title.
tb.send_audio(CHAT_ID, file_data, 1, 'eternnoir', 'pyTelegram')

# sendVoice
voice = open('/tmp/voice.ogg', 'rb')
tb.send_voice(chat_id, voice)
tb.send_voice(chat_id, "FILEID")

# sendDocument
doc = open('/tmp/file.txt', 'rb')
tb.send_document(chat_id, doc)
tb.send_document(chat_id, "FILEID")

# sendSticker
sti = open('/tmp/sti.webp', 'rb')
tb.send_sticker(chat_id, sti)
tb.send_sticker(chat_id, "FILEID")

# sendVideo
video = open('/tmp/video.mp4', 'rb')
tb.send_video(chat_id, video)
tb.send_video(chat_id, "FILEID")

# sendLocation
tb.send_location(chat_id, lat, lon)

# sendChatAction
# action_string can be one of the following strings: 'typing', 'upload_photo', 'record_video', 'upload_video',
# 'record_audio', 'upload_audio', 'upload_document' or 'find_location'.
tb.send_chat_action(chat_id, action_string)

# getFile
# Downloading a file is straightforward
# Returns a File object
import requests
file_info = tb.get_file(file_id)

file = requests.get('https://api.telegram.org/file/bot{0}/{1}'.format(API_TOKEN, file_info.file_path))

# Клавиатура

Можно отправлять пользователю специальную клавиатуру для ответа или интерактивные кнопки. Клавиатуру можно добавить во все функции, которые начинаются со слова `send_` например так: `tb.send_message(chat_id, text, reply_markup=keyboard)`. При этом `keyboard` - это объект `ReplyKeyboardMarkup`, `ReplyKeyboardRemove` или `ForceReply`. Рассмотрим `ReplyKeyboardMarkup`:

In [ ]:
from telebot import types

# создаем клавиатуру
keyboard = types.ReplyKeyboardMarkup(row_width=2)  # row_width говорит сколько кнопок должно поместиться в одном ряду
# добавляем кнопки
btn1 = types.KeyboardButton('да')
btn2 = types.KeyboardButton('нет')
btn3 = types.KeyboardButton('не знаю')
keyboard.add(btn1, btn2, btn3)
# отправляем
tb.send_message(chat_id, "Вы студент?", reply_markup=keyboard)


# Можно создавать каджый ряд отдельно
# создаем клавиатуру 
keyboard = types.ReplyKeyboardMarkup()  # здесь нет row_width !!!
# создаем кнопки
btn1 = types.KeyboardButton('red')
btn2 = types.KeyboardButton('blue')
btn3 = types.KeyboardButton('green')
btn4 = types.KeyboardButton('brown')
btn5 = types.KeyboardButton('black')
# добавляем ряды кнопок в клавиатуру
keyboard.row(btn1, btn2, btn3)
keyboard.row(bnt4, btn5)
# отправляем
tb.send_message(chat_id, "Choose color:", reply_markup=keyboard)

[Больше про кнопки в документации](https://github.com/eternnoir/pyTelegramBotAPI#reply-markup) и в [туториале](https://groosha.gitbooks.io/telegram-bot-lessons/content/chapter8.html)

# Пример бота: разметка

Предположим у вас есть тысячи отзывов на кинофильмы, и вам нужно их разметить как "положительные", "отрицательные" или "нейтральные". Вы можете сделать бота, который будет отправлять юзеру какой-то отзыв из нашего списка и клавиатуру с кнопками "+", "-", "=", а затем бот будет записывать ответы. Бота мы можем отправить друзьям, чтобы они помогли нам с разметкой=)

(строчки, связанные с фласком, в коде ниже отсутствуют)
Смотрим пример:

In [ ]:
# -*- coding: utf-8 -*-
# импортируем модули
import telebot
import conf
import random
import shelve
from telebot import types

bot = telebot.TeleBot(conf.TOKEN, threaded=False)

# предположим, отзывы у нас хранятся в виде csv-файла с номерами отзывов и собственно текстами
with open('/home/2016learnpython/myapp/reviews.csv', 'r', encoding='utf-8') as f:
    reviews = {}  # создадим словать отзывов
    for line in f:
        num, text = line.strip().split('\t')
        reviews[num] = text
review_keys = list(reviews.keys())  # и отдельно массив ключей


# собираем клавиатуру для разметки (возможно имеет смысл добавить кнопку "не знаю"?)
keyboard = types.ReplyKeyboardMarkup(row_width=3)
btn1 = types.KeyboardButton('+')
btn2 = types.KeyboardButton('-')
btn3 = types.KeyboardButton('=')
keyboard.add(btn1, btn2, btn3)


# shelve используется как мини-база данных, там можно хранить файлы в виде "ключ-значение"
# в этой базе мы будем хранить информацию о том, какой отзыв мы недавно прислали юзеру
shelve_name = '/home/USERNAME/myapp/shelve.db'  # Файл с хранилищем


def set_user_review(chat_id, review):
    """
    Записываем юзера в базу данных и запоминаем номер отзыва, который мы ему дали
    """
    with shelve.open(shelve_name) as storage:
        storage[str(chat_id)] = review


def get_user_review(chat_id):
    """
    Вспоминаем, какой отзыв мы отправили на разметку
    :return: (str) Номер отзыва / None
    """
    with shelve.open(shelve_name) as storage:
        try:
            review = storage[str(chat_id)]
            return review
        # Если человека нет в базе, ничего не возвращаем
        except KeyError:
            return None

# этот обработчик запускает функцию send_welcome, когда пользователь отправляет команду /help
@bot.message_handler(commands=['help'])
def send_welcome(message):
    bot.send_message(message.chat.id, "Здравствуйте! Это бот для разметки отзывов на кинофильмы.\n Положительные отзывы отмечаются плюсом +, отрицательные минусом -, а нейтральные знаком равно =.")


# этот обработчик запускает функцию, когда пользователь отправляет команду /start
@bot.message_handler(commands=['start'])
def send_first_review(message):
    review_num = random.choice(review_keys)
    bot.send_message(message.chat.id, reviews[review_num], reply_markup=keyboard)
    set_user_review(message.chat.id, review_num)


@bot.message_handler(regexp='[-+=]')  # этот обработчик реагирует на символы разметки и записывает результаты
def get_answer(message):
    review_num = get_user_review(message.chat.id)  # проверяем, есть ли юзер в базе данных
    if review_num:
        # если есть, открываем файл с результатами и записываем туда разметку
        with open('/home/USERNAME/myapp/results.csv', 'a', encoding='utf-8') as results:
            results.write(review_num + '\t' + message.text + '\n')
        # и сразу отправляем новый отзыв
        review_num = random.choice(review_keys)
        bot.send_message(message.chat.id, reviews[review_num], reply_markup=keyboard)
        set_user_review(message.chat.id, review_num)
    else:
        # если нет, то что-нибудь говорим об этом
        bot.send_message(message.chat.id, 'Вы не разметили отзыв.')


## Задания
1) Попробуйте запустить этого бота у себя на компьютере.

2) Добавьте функции, которые позволят пользователю завершить разметку по команде "`/stop`" и удалиться из базы данных.

3) Попробуйте сделать так, чтобы пользователю отправлялись только те отзывы, которые еще не были размечены ни разу.